In [16]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
from experiment import experiment

In [18]:
import os

In [19]:
import pandas as pd

In [20]:
results = os.listdir('results')

In [21]:
results = [result for result in results if result.endswith('.pkl') and "6.05.2023" in result and "rfqr_20runs" in result and "std" in result and "B=100" in result and "logTrue" in result]

In [22]:
results

['bio_rfqr_20runs_0.25fraction_logTrue_randomConformalTrue_cqrbaggstd_twosidedconformalFalse_B=100_06.05.2023.pkl',
 'star_rfqr_20runs_1fraction_logTrue_randomConformalTrue_cqrbaggstd_twosidedconformalFalse_B=100_06.05.2023.pkl',
 'community_rfqr_20runs_1fraction_logTrue_randomConformalTrue_cqrbaggstd_twosidedconformalFalse_B=100_06.05.2023.pkl',
 'concrete_rfqr_20runs_1fraction_logTrue_randomConformalTrue_cqrbaggstd_twosidedconformalFalse_B=100_06.05.2023.pkl',
 'bike_rfqr_20runs_1fraction_logTrue_randomConformalTrue_cqrbaggstd_twosidedconformalFalse_B=100_06.05.2023.pkl',
 'homes_rfqr_20runs_0.5fraction_logTrue_randomConformalTrue_cqrbaggstd_twosidedconformalFalse_B=100_06.05.2023.pkl']

In [23]:
output = pd.DataFrame(columns=['UACQR-P','UACQR-S', 'CQR','CQR-r', 'UACQR-P SE', 'UACQR-S SE', 'CQR SE','CQR-r SE'])
for i,result in enumerate(results):
    title = results[i].split('_')[0] if results[i].split('_')[0] not in ["facebook","meps"] else '_'.join(results[i].split('_')[0:2])
    result = experiment(file_name='results/'+results[i])
    result.plot(calc_only=True)
    output.loc[title,['UACQR-P','UACQR-S', 'CQR','CQR-r']] = result.mean_results.min()
    output.loc[title,['UACQR-P SE','UACQR-S SE', 'CQR SE','CQR-r SE']] = result.sem_results.stack().loc[pd.MultiIndex.from_frame(result.mean_results.idxmin().rename('min_samples_leaf').reset_index().iloc[:,::-1])].to_numpy()
    
    

In [24]:
output

,UACQR-P,UACQR-S,CQR,CQR-r,UACQR-P SE,UACQR-S SE,CQR SE,CQR-r SE
bio,1.513867,1.630464,1.631176,1.629646,0.006304,0.003856,0.00387,0.003794
star,0.174554,0.174126,0.17392,0.174045,0.000938,0.000747,0.000721,0.000766
community,0.268256,0.314898,0.31572,0.315571,0.003129,0.002718,0.002461,0.002477
concrete,0.729421,0.80676,0.841803,0.795446,0.010338,0.007609,0.007359,0.007996
bike,1.613055,1.901252,1.945633,1.885768,0.006845,0.007044,0.007634,0.007185
homes,0.59591,0.635026,0.650137,0.634358,0.002667,0.0036,0.003086,0.003437


In [25]:
from functools import partial

In [26]:
def format_fig_and_se(row, method='CQR'):
    fig = row[method]
    se = row[method+' SE']
    if fig >1000:
        return f"{fig:.0f} ({se:.0f})"
    elif fig >1:
        return f"{fig:.3f} ({se:.3f})"
    else:
        return f"{fig:.3f} ({se:.3f})"

In [27]:
drop_columns = [c for c in output.columns if 'SE' in c]

In [28]:
output['CQR'] = output.apply(partial(format_fig_and_se, method='CQR'), axis=1)
output['UACQR-S'] = output.apply(partial(format_fig_and_se, method='UACQR-S'), axis=1)
output['UACQR-P'] = output.apply(partial(format_fig_and_se, method='UACQR-P'), axis=1)
output['CQR-r'] = output.apply(partial(format_fig_and_se, method='CQR-r'), axis=1)

In [29]:
output.drop(columns=drop_columns, inplace=True)

In [30]:
# Export to LaTeX
latex_table = output.sort_index().style.to_latex()


print('\_'.join(latex_table.split('_')))

\begin{tabular}{lllll}
 & UACQR-P & UACQR-S & CQR & CQR-r \\
bike & 1.613 (0.007) & 1.901 (0.007) & 1.946 (0.008) & 1.886 (0.007) \\
bio & 1.514 (0.006) & 1.630 (0.004) & 1.631 (0.004) & 1.630 (0.004) \\
community & 0.268 (0.003) & 0.315 (0.003) & 0.316 (0.002) & 0.316 (0.002) \\
concrete & 0.729 (0.010) & 0.807 (0.008) & 0.842 (0.007) & 0.795 (0.008) \\
homes & 0.596 (0.003) & 0.635 (0.004) & 0.650 (0.003) & 0.634 (0.003) \\
star & 0.175 (0.001) & 0.174 (0.001) & 0.174 (0.001) & 0.174 (0.001) \\
\end{tabular}

